In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import collections
import os
import random
import numpy as np
from tqdm import tqdm
import sys, email
import pandas as pd
import math


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
ENRON_EMAIL_DATASET_PATH = 'enron.csv'

# load enron dataset
import pandas as pd

emails_df = pd.read_csv(ENRON_EMAIL_DATASET_PATH)
print(emails_df.shape)
emails_df.head()


(517401, 8)


,Message-ID,Date,From,To,Subject,Message,Cc,Bcc
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,Here is our forecast\n\n \n,NaN,NaN
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,Traveling to have a business meeting takes the...,NaN,NaN
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,test successful. way to go!!!\n,NaN,NaN
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,NaN,"Randy,\n\n Can you send me a schedule of the s...",NaN,NaN
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Let's shoot for Tuesday at 11:45. \n,NaN,NaN


In [3]:
emails_sample_df = emails_df.copy()

In [4]:
# clean up subject line
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.lower()
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace('re:', ' ')
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace('fw:', ' ')
#emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'[^a-z]', ' ')
emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'\s+', ' ')

# clean up content line
emails_sample_df['Message'] = emails_sample_df['Message'].str.lower()
emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'[^a-z]', ' ')
emails_sample_df['Message'] = emails_sample_df['Message'].str.replace('fw:', ' ')
emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'\s+', ' ')



C:\Users\Aaron\AppData\Local\Temp/ipykernel_30664/321775432.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Subject'] = emails_sample_df['Subject'].str.replace(r'\s+', ' ')
C:\Users\Aaron\AppData\Local\Temp/ipykernel_30664/321775432.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'[^a-z]', ' ')
C:\Users\Aaron\AppData\Local\Temp/ipykernel_30664/321775432.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  emails_sample_df['Message'] = emails_sample_df['Message'].str.replace(r'\s+', ' ')


In [5]:
emails_sample_df.head()

,Message-ID,Date,From,To,Subject,Message,Cc,Bcc
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,here is our forecast,NaN,NaN
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,,traveling to have a business meeting takes the...,NaN,NaN
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,test,test successful way to go,NaN,NaN
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,NaN,randy can you send me a schedule of the salary...,NaN,NaN
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,hello,let s shoot for tuesday at,NaN,NaN


In [6]:
emails_sample_df.head()

,Message-ID,Date,From,To,Subject,Message,Cc,Bcc
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,here is our forecast,NaN,NaN
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,,traveling to have a business meeting takes the...,NaN,NaN
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,test,test successful way to go,NaN,NaN
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,NaN,randy can you send me a schedule of the salary...,NaN,NaN
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,hello,let s shoot for tuesday at,NaN,NaN


In [7]:
# create sentence list
emails_text = (emails_sample_df["Subject"] + ". " + emails_sample_df["Message"]).tolist()

In [8]:
emails_text

[nan,
 ' . traveling to have a business meeting takes the fun out of the trip especially if you have to prepare a presentation i would suggest holding the business plan meetings here then take a trip without any formal business meetings i would even try and get some honest opinions on whether a trip is even desired or necessary as far as the business meetings i think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not too often the presenter speaks and the others are quiet just waiting for their turn the meetings might be better if held in a round table discussion format my suggestion for where to go is austin play golf and rent a ski boat and jet ski s flying somewhere takes too much time ',
 ' test. test successful way to go ',
 nan,
 ' hello. let s shoot for tuesday at ',
 ' hello. greg how about either next tuesday or thursday phillip ',
 nan,
 ' prc review - phone calls. any morning between and ',
 ' high s

In [9]:
sentences = ' '.join([str(i) for i in emails_text])
words = sentences.split()

print('Data size', len(words))


Data size 141005095


In [10]:
# get unique words and map to glove set
print('Unique word count', len(set(words))) 

Unique word count 588751


In [11]:
# drop rare words
vocabulary_size = 10000

In [12]:
def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in tqdm(words):
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)

100%|███████████████████████████████████████████████████████████████| 141005095/141005095 [00:50<00:00, 2776906.31it/s]


In [13]:
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 10063222], ('the', 5605630), ('to', 4006328), ('and', 2555652), ('of', 2358090)]
Sample data [965, 1260, 4047, 2, 25, 5, 104, 135, 1647, 1] ['nan', '.', 'traveling', 'to', 'have', 'a', 'business', 'meeting', 'takes', 'the']


In [14]:
data_index = 0
 
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

In [15]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

1260 . -> 965 nan
1260 . -> 4047 traveling
4047 traveling -> 1260 .
4047 traveling -> 2 to
2 to -> 4047 traveling
2 to -> 25 have
25 have -> 2 to
25 have -> 5 a


In [16]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 300  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.


In [17]:
graph = tf.Graph()


In [18]:
with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [19]:
# Step 5: Begin training.
num_steps = 200001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in range(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  211.47381591796875
Nearest to all: submission, social, surcharge, weren, when, ashford, mastercard, phone,
Nearest to in: brennan, sai, coverage, founder, side, mobile, peco, acquisitions,
Nearest to about: joann, jane, decoration, trigger, imperative, identified, believes, elink,
Nearest to sent: urgent, kaiser, fruit, likely, brother, youngblood, financially, heights,
Nearest to have: warn, trademarks, failure, consider, ckm, husser, cmu, hard,
Nearest to http: training., rhonda, rate, voters, breathitt, cfm, ehs, penman,
Nearest to re: natalie, parties, breaker, houind, spokesman, trained, masson, indirectly,
Nearest to up: trades, killing, whitecase, things, coordinators, catching, powerplant, traveling,
Nearest to as: afford, epenergy, convention, remi, water, buddy, appellate, published,
Nearest to that: springs, work, sharron, sandherr, delainey, employer, adviser, sdge,
Nearest to c: justice, intends, facing, thereby, utilizing, trang, fam

Average loss at step  52000 :  2.1473828180544077
Average loss at step  54000 :  2.7291635892242194
Average loss at step  56000 :  3.6694815253019333
Average loss at step  58000 :  3.7263473778367042
Average loss at step  60000 :  3.7395607566833498
Nearest to all: std, social, rtos, learned, submission, when, gorny, ashford,
Nearest to in: std, on, of, with, for, liabilities, to, at,
Nearest to about: decoration, elink, paris, jonesday, next, trigger, of, invests,
Nearest to sent: elink, enron, urgent, ect, fruit, std, kaiser, on,
Nearest to have: be, has, are, had, ve, need, std, pacificorp,
Nearest to http: std, pm, soblander, click, resistance, ect, image, ees,
Nearest to re: are, natalie, rtf, subject, spokesman, indirectly, parties, meta,
Nearest to up: trades, coordinators, reschedule, turns, resistance, importance, std, rangel,
Nearest to as: but, if, std, senators, td, in, that, asap,
Nearest to that: it, which, std, he, this, UNK, and, i,
Nearest to c: d, justice, thereby, fa

Average loss at step  120000 :  4.115632204025983
Nearest to all: std, enron, resistance, these, vnf, rtos, to, social,
Nearest to in: on, of, std, with, from, resistance, at, for,
Nearest to about: on, elink, from, decoration, yesterday, thrown, jane, believes,
Nearest to sent: cc, ect, youngblood, elink, maviglio, urgent, acquired, fruit,
Nearest to have: has, had, ve, be, need, are, get, want,
Nearest to http: pm, www, std, flip, hotmail, soblander, click, cd,
Nearest to re: are, subject, natalie, euenronxgate, parties, do, breaker, edelman,
Nearest to up: out, down, reschedule, resistance, principal, trades, turns, success,
Nearest to as: and, in, std, UNK, if, when, but, how,
Nearest to that: which, but, what, it, enron, this, and, for,
Nearest to c: justice, thereby, sits, b, families, UNK, refunds, asp,
Nearest to t: not, must, to, you, normally, colwell, resistance, rtf,
Nearest to corp: na, com, pg&e, pts, kong, std, established, hou,
Nearest to get: have, take, be, hell, want

Average loss at step  182000 :  3.6362058849930765
Average loss at step  184000 :  3.8763181319832802
Average loss at step  186000 :  3.824316010773182
Average loss at step  188000 :  3.6606734637916087
Average loss at step  190000 :  3.6899373546242713
Nearest to all: these, rtos, what, std, those, and, the, ashford,
Nearest to in: on, std, of, for, to, at, from, UNK,
Nearest to about: on, from, with, that, elink, of, know, when,
Nearest to sent: cc, ect, UNK, subject, mailto, pm, com, enronxgate,
Nearest to have: had, has, ve, be, are, need, were, been,
Nearest to http: hotmail, image, www, click, std, the, and, please,
Nearest to re: fw, are, subject, euenronxgate, edelman, natalie, breaker, wild,
Nearest to up: out, down, off, me, turns, him, back, principal,
Nearest to as: if, and, when, std, how, that, but, in,
Nearest to that: which, but, what, he, it, when, if, as,
Nearest to c: b, d, sfgate, t, robert, asp, unlike, justice,
Nearest to t: not, colwell, d, really, you, identific

In [20]:
# create a dataframe using the embedded vectors and attach the key word as row header
import pandas as pd
enrond_dataframe = pd.DataFrame(final_embeddings)
enrond_dataframe = enrond_dataframe.set_index([list(reverse_dictionary.values())])


In [21]:
# See what it learns and look at clusters to pull out major themes in the data
CLUSTER_SIZE = 300 
# cluster vector and investigate top groups
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=CLUSTER_SIZE)
cluster_make = kmeans.fit_predict(enrond_dataframe)

labels = kmeans.predict(enrond_dataframe)
import collections
cluster_frequency = collections.Counter(labels)
print(cluster_frequency)
cluster_frequency.most_common()

clusters = {}
n = 0
for item in labels:
    if item in clusters:
        clusters[item].append(list(enrond_dataframe.index)[n])
    else:
        clusters[item] = [list(enrond_dataframe.index)[n]]
    n +=1

for k,v in cluster_frequency.most_common(100):
  print('\n\n')
  print('Cluster:', k)
  print (' '.join(clusters[k]))

Counter({62: 258, 248: 233, 286: 170, 23: 167, 291: 118, 266: 106, 167: 105, 37: 100, 142: 100, 18: 98, 288: 93, 272: 92, 169: 91, 26: 84, 194: 83, 253: 82, 188: 77, 159: 76, 63: 75, 165: 70, 210: 69, 292: 69, 116: 69, 96: 67, 121: 67, 140: 64, 134: 62, 103: 62, 283: 60, 229: 60, 10: 55, 230: 55, 124: 55, 11: 55, 110: 54, 264: 53, 181: 52, 108: 51, 85: 49, 39: 48, 277: 47, 105: 47, 129: 47, 153: 47, 279: 46, 127: 46, 220: 45, 186: 45, 197: 44, 207: 44, 99: 44, 41: 43, 276: 43, 212: 42, 145: 42, 118: 42, 81: 41, 232: 41, 235: 41, 254: 41, 163: 40, 135: 40, 251: 40, 189: 39, 77: 38, 158: 38, 50: 38, 241: 38, 3: 38, 65: 37, 33: 37, 160: 36, 252: 36, 299: 36, 259: 36, 76: 36, 260: 36, 176: 36, 208: 35, 80: 35, 91: 35, 201: 35, 120: 34, 131: 34, 297: 34, 271: 34, 2: 34, 70: 34, 187: 34, 128: 34, 195: 34, 0: 34, 68: 34, 52: 34, 294: 33, 34: 33, 107: 33, 204: 33, 177: 32, 137: 32, 293: 32, 275: 32, 213: 32, 280: 32, 269: 32, 203: 31, 206: 31, 40: 31, 171: 30, 12: 30, 182: 30, 224: 30, 172: 30